<a href="https://colab.research.google.com/github/ybchen97/filler_detection/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model

Syncing from google drive and github... for more info on this code, refer [here](https://zerowithdot.com/colab-github-workflow/)

In [25]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/filler_detection/train_data/XY_Train'       # path to your project on Drive
PROJ2= 'My Drive/filler_detection/train_data/dev_npy'

drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
PROJCT_PATH2 = join(ROOT, PROJ2)
!mkdir "{PROJECT_PATH}"     # in case we haven't created it already   
!mkdir "{PROJECT_PATH2}"

GIT_PATH = "https://github.com/ybchen97/filler_detection.git"
!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/filler_detection/train_data/XY_Train’: File exists
fatal: destination path 'filler_detection' already exists and is not an empty directory.
mv: cannot stat './temp/*': No such file or directory


Install packages in this local notebook specified in requirements.txt

In [4]:
!pip install -r '/content/filler_detection/requirements.txt'

     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 788kB 15.0MB/s 
     |████████████████████████████████| 378kB 21.9MB/s 
     |████████████████████████████████| 12.4MB 20.8MB/s 
     |████████████████████████████████| 20.2MB 158kB/s 
     |████████████████████████████████| 358kB 64.3MB/s 
     |████████████████████████████████| 1.3MB 62.3MB/s 
     |████████████████████████████████| 921kB 45.9MB/s 
     |████████████████████████████████| 276kB 55.6MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 3.9MB 53.0MB/s 
     |████████████████████████████████| 421.8MB 40kB/s 
     |████████████████████████████████| 450kB 51.5MB/s 
     |████████████████████████████████| 133kB 76.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=6f3bee0f3114409b3978363523548eb596a36fa93f78f39f28a71be9506fbdbc
  Stored in directo

Importing and setting up env variables...

In [6]:
import numpy as np
from pydub import AudioSegment
from pydub.playback import play
import random
import sys
import io
import os
import glob
import IPython
import wave
import pylab
from tf_utils import *
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile

# Import files for trigger-word detection model
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/filler_detection/train_data/XY_Train'       # path to your project on Drive

DATA_PATH = join(ROOT, PROJ)
REPO = "/content/filler_detection/"

In [0]:
POSITIVE_DIRECTORY = "./raw_data/positive_data/"
BACKGROUND_DIRECTORY = "./raw_data/background_data/"
NEGATIVES_DIRECTORY = "./raw_data/google_dataset/"
NEGATIVES_TRUNCATED_DIRECTORY = "./raw_data/google_dataset_truncated/"
AUDIO_EXAMPLES_DIRECTORY = "./audio_examples/"
POSITIVE_EXAMPLE = "jh_1.wav"
AUDIO_EXAMPLE = "example_train.wav"

The fun begins...
## Model Loading

In [0]:
# GRADED FUNCTION: model

def model(input_shape):
    """
    Function creating the model's graph in Keras library.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)
    
    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
    
    # Step 1: CONV Layer
    # CONV-1D
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    # Batch Normalization
    X = BatchNormalization()(X)
    # RelU activation
    X = Activation("relu")(X)
    # Dropout (using rate 0.8)
    X = Dropout(rate=0.8)(X)
    
    # Step 2: First GRU Layer
    # GRU (use 128 units to return the sequences)
    X = GRU(units=128, return_sequences=True)(X)
    # Dropout (using rate 0.8)
    X = Dropout(rate=0.8)(X)
    # Batch Normalization
    X = BatchNormalization()(X)
    
    # Step 3: Second GRU Layer
    # GRU (use 128 units to return the sequences)
    X = GRU(units=128, return_sequences=True)(X)
    # Dropout (using rate 0.8)
    X = Dropout(rate=0.8)(X)
    # Batch Normalization
    X = BatchNormalization()(X)
    # Dropout (using rate 0.8)
    X = Dropout(rate=0.8)(X)
    
    # Step 4: Time-distributed dense layer
    X = TimeDistributed(Dense(1, activation="sigmoid"))(X)
    
    # Return model
    model = Model(inputs = [X_input], outputs = X)
    return model

Sanity check for `Tx`, `n_freq`, `Ty`. 

1. Input into model `Tx` and `n_freq`
2. Call `model.summary()`
3. `Tx` of **sample** and variable = `input_7.shape[1]` (ie column 2, row 1, second element of array)
4. `Ty` = `input_7.shape[1]` (ie column 2, row 2, second element of array)

In [0]:
Tx = 5490 # The number of time steps input to the model from the spectrogram
n_freq = 129 # Number of frequencies input to the model at each time step of the spectrogram
Ty = 1369 # The number of time steps in the output of our model

### Load Pre-trained Model

In [10]:
model = load_model("filler_detection/trained_model.h5")
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5490, 129)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1369, 196)         379456    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1369, 196)         784       
_________________________________________________________________
activation_1 (Activation)    (None, 1369, 196)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1369, 196)         0         
_________________________________________________________________
gru_1 (GRU)                  (None, 1369, 128)         124800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1369, 128)         0   

### Create New Model (if no pre-trained)

In [7]:
model = model(input_shape = (Tx, n_freq))
model.summary()

NameError: ignored

## Model Training

In [0]:
# Function to fit and further train the model
def model_train(model, X, Y):
    """
    Function to train the model further using Adam optimiser and binary 
    cross entropy loss.
    
    Arguments:
    model -- Model to train
    X -- X data to train on
    Y -- Y data to train on
    """
    
    opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(X, Y, batch_size=5, epochs=1)

In [13]:
# Train the model on stubbed data downloaded from Coursera
# model = load_model(STUB_MODEL)

X = np.load("/content/drive/My Drive/filler_detection/train_data/XY_Train/X_1.npy")
Y = np.load("/content/drive/My Drive/filler_detection/train_data/XY_Train/Y_1.npy")

model_train(model, X, Y)

Epoch 1/1
1000/1000 [==============================] - 799s 799ms/step - loss: 0.8737 - accuracy: 0.6759


## Model Testing

In [26]:
print(f"{DATA_PATH}/Y_2.npy")
print(f"{PROJECT_PATH}/Y_2.npy")

/content/drive/My Drive/filler_detection/train_data/XY_Train/Y_2.npy
/content/drive/My Drive/filler_detection/train_data/XY_Train/Y_2.npy


In [27]:
# Function to test the model on new data
def model_test(model, X_dev, Y_dev):
    loss, acc = model.evaluate(X_dev, Y_dev)
    print("Dev set accuracy = ", acc)

X_dev = np.load(f"{PROJECT_PATH}/X_2.npy")
Y_dev = np.load(f"{PROJECT_PATH}/Y_2.npy")
print("X: {} Y: {}".format(X_dev.shape, Y_dev.shape))

X: (1000, 5490, 129) Y: (1000, 1369, 1)


In [28]:
model_test(model, X_dev, Y_dev)

1000/1000 [==============================] - 49s 49ms/step
Dev set accuracy =  0.9360423684120178


In [47]:
X_11 = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_11X.npy")
Y_11 = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_11Y.npy")
print("X: {} Y: {}".format(X_11.shape, Y_11.shape))

X_12 = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_12X.npy")
Y_12 = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_12Y.npy")
print("X: {} Y: {}".format(X_12.shape, Y_12.shape))

X_13 = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_13X.npy")
Y_13 = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_13Y.npy")
print("X: {} Y: {}".format(X_13.shape, Y_13.shape))

X_14 = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_14X.npy")
Y_14 = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_14Y.npy")
print("X: {} Y: {}".format(X_14.shape, Y_14.shape))








X: (1, 5490, 129) Y: (1, 1369, 1)
X: (1, 5490, 129) Y: (1, 1369, 1)
X: (1, 5490, 129) Y: (1, 1369, 1)
X: (1, 5490, 129) Y: (1, 1369, 1)


In [48]:
model_test(model, X_11, Y_11)
model_test(model, X_12, Y_12)
model_test(model, X_13, Y_13)
model_test(model, X_14, Y_14)

1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9269539713859558
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9627465009689331
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9634770154953003
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9634770154953003


In [46]:
list_of_dev_ids = [11,12,13,14,20,21,22,23,24,30,31,32,33,34,35,36,37,38,39,310,40,41,42,43,44]
for id in list_of_dev_ids:
    X_id = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_" + str(id) + "X.npy")

    Y_id = np.load("/content/drive/My Drive/filler_detection/train_data/dev_npy/cont_" + str(id) + "Y.npy")
    print(("X" + str(id) + ": {} Y" + str(id) + ": {}").format(X_id.shape, Y_id.shape))
    model_test(model, X_id, Y_id)

model_test(model, X_11, Y_11)


X11: (1, 5490, 129) Y11: (1, 1369, 1)
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9269539713859558
X12: (1, 5490, 129) Y12: (1, 1369, 1)
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9627465009689331
X13: (1, 5490, 129) Y13: (1, 1369, 1)
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9634770154953003
X14: (1, 5490, 129) Y14: (1, 1369, 1)
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9634770154953003
X20: (1, 5490, 129) Y20: (1, 1369, 1)
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9634770154953003
X21: (1, 5490, 129) Y21: (1, 1369, 1)
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9634770154953003
X22: (1, 5490, 129) Y22: (1, 1369, 1)
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9634770154953003
X23: (1, 5490, 129) Y23: (1, 1369, 1)
1/1 [==============================] - 1s 1s/step
Dev set accuracy =  0.9